In [4]:
!pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp313-cp313-win_amd64.whl.metadata (60 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Using cached numpy-2.2.6-cp313-cp313-win_amd64.whl (12.6 MB)

   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.61.0 requires numpy<2.2,>=1.24, but you have numpy 2.2.6 which is incompatible.
streamlit 1.45.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.


In [5]:
!pip install pytesseract

In [43]:
import pandas as pd
import numpy as np
import cv2
# from skimage import color # 이 함수에서는 color 모듈을 사용하지 않습니다.

# 1. 히트맵 및 수치 범위 정의
HEATMAP_PATH = 'heatmap_2.PNG' # 파일 경로가 정확한지 다시 확인해주세요.
EXCEL_FILENAME = "heatmap_quantified_data_53x25_final.xlsx"

N_ROWS = 53  # 행 (Y축)
N_COLS = 25  # 열 (X축)
# 이 값들은 클러스터링 함수에서 직접 사용되지는 않지만, 최종 클리핑 범위를 나타냅니다.
MIN_VALUE = -2.0 
MAX_VALUE = 4.0  
CENTER_VALUE = 0.0

# 2. 색상 클러스터링 기반 변환 함수 (5단계 세분화)
def rgb_to_value_final_5step(r_norm, g_norm, b_norm):
    """
    고객님의 요청에 따라 4~2, 2~0, 0~-2 범위를 5단계로 세분화하여 매핑합니다.
    """
    
    # R과 G의 차이를 사용하여 색조(Hue)를 근사합니다.
    # B 채널이 낮을수록 따뜻한 색(양수)입니다.
    
    # 1. 빨강 (4.0 ~ 3.0)
    if r_norm > 0.6 and g_norm < 0.4:
        # R이 매우 높고 G는 낮음 -> 진한 빨강
        # (R-G) 차이가 클수록 4.0에 가까움
        value = 3.0 + (r_norm - 0.6) * 2.5 # 0.6에서 4.0으로 매핑 시작
        return np.clip(value, 3.0, 4.0)

    # 2. 주황 (3.0 ~ 2.0)
    elif r_norm > 0.4 and g_norm >= 0.4 and b_norm < 0.3:
        # R과 G가 모두 높지만 B는 낮음 -> 주황/옅은 빨강
        # R과 G의 차이 또는 B의 강도에 따라 3.0에서 2.0으로 떨어짐
        value = 3.0 - b_norm * (1.0 / 0.3)
        return np.clip(value, 2.0, 3.0)

    # 3. 노랑 ~ 연두 (2.0 ~ 1.0)
    elif g_norm > r_norm * 0.9 and b_norm < 0.5:
        # G가 R과 비슷하거나 높고 B가 중간 -> 노랑, 연두색 (0에 가까움)
        # B가 0.3이면 2.0, B가 0.5이면 1.0에 가깝도록 조정
        value = 3.5 - b_norm * (2.5) # 2.0과 1.0 사이로 매핑
        return np.clip(value, 1.0, 2.0)
        
    # 4. 하늘색 (1.0 ~ 0.0)
    elif b_norm >= 0.5 and (r_norm + g_norm) / 2 < b_norm:
        # B가 명확히 R, G보다 강함 -> 하늘색/밝은 파랑
        # B가 0.5이면 1.0, B가 0.7이면 0.0에 가깝도록 조정
        value = 1.0 - (b_norm - 0.5) * (1.0 / 0.2)
        return np.clip(value, 0.0, 1.0)
        
    # 5. 진한 파랑 (0.0 ~ -2.0)
    else:
        # B가 압도적으로 강함 -> 진한 파랑
        # B가 0.7이면 0.0, B가 1.0이면 -2.0에 가깝도록 조정
        value = 0.0 - (b_norm - 0.7) * (2.0 / 0.3)
        return np.clip(value, -2.0, 0.0)

# **메인 코드의 quantify_heatmap 함수 내에서 이 함수(rgb_to_value_final_5step)를 호출하도록 수정 후 실행해야 합니다.**

# 3. 이미지 분석 및 수치 추출 메인 함수
def quantify_heatmap(image_path, rows, cols, vmin, vmax, center):
    
    quantified_data = np.zeros((rows, cols))
    
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {image_path}")
            
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # ⚠️ [가장 중요한 수동 조정] 픽셀 좌표 설정 ⚠️
        # 이 좌표는 이미지에 맞게 수동으로 조정해야 합니다!
        x_start = 28  
        x_end = 478   
        y_start = 28  
        y_end = 650   
        # ----------------------------------------------------
        
        cropped_img = img_rgb[y_start:y_end, x_start:x_end]
        cropped_height, cropped_width, _ = cropped_img.shape
        
        cell_height = cropped_height // rows
        cell_width = cropped_width // cols
        
        # 각 셀을 순회하며 평균 색상 추출 및 값 변환
        for i in range(rows):
            for j in range(cols):
                # 그리드 선 제외를 위한 샘플링 비율 조정 가능 (0.15 대신 0.20 등)
                border_trim_y = int(cell_height * 0.15)
                border_trim_x = int(cell_width * 0.15)

                sampled_area = cropped_img[
                    i * cell_height + border_trim_y : (i + 1) * cell_height - border_trim_y,
                    j * cell_width + border_trim_x : (j + 1) * cell_width - border_trim_x
                ]
                
                avg_rgb_255 = np.mean(sampled_area, axis=(0, 1))
                r, g, b = avg_rgb_255 / 255.0 # 0~1로 정규화
                
                # 📢 클러스터링 함수 호출 (vmin, vmax, center 인자 제거)
                value = rgb_to_value_clustering(r, g, b)
                quantified_data[i, j] = value
        
    except Exception as e:
        print(f"❌ 이미지 분석 또는 파일 처리 중 오류 발생: {e}")
        return np.zeros((rows, cols))
        
    return quantified_data


# 4. 수치화 실행 및 엑셀 저장
# vmin, vmax, center는 quantify_heatmap 함수 내에서 사용되지 않지만,
# 함수 인자 형태 유지를 위해 그대로 전달합니다.
final_data = quantify_heatmap(HEATMAP_PATH, N_ROWS, N_COLS, MIN_VALUE, MAX_VALUE, CENTER_VALUE)

if final_data.any():
    df_result = pd.DataFrame(final_data).round(3)
    try:
        df_result.to_excel(EXCEL_FILENAME, index=False, header=False)
        print(f"\n🎉 **최종 데이터가 '{EXCEL_FILENAME}' 파일로 저장되었습니다.**")
        print(f"저장된 데이터 크기: {N_ROWS}행, {N_COLS}열")
    except Exception as e:
        print(f"❌ 엑셀 파일 저장 중 오류가 발생했습니다: {e}")


🎉 **최종 데이터가 'heatmap_quantified_data_53x25_final.xlsx' 파일로 저장되었습니다.**
저장된 데이터 크기: 53행, 25열


In [45]:
import pandas as pd
import numpy as np
import cv2
# from skimage import color # 이 함수에서는 color 모듈을 사용하지 않습니다.

# 1. 히트맵 및 수치 범위 정의
# 💡 파일 확장자 확인! 실제 파일 이름이 'heatmap_2.PNG'가 아니라면 수정하세요.
HEATMAP_PATH = 'heatmap_2.PNG' 
EXCEL_FILENAME = "heatmap_qua.xlsx"

N_ROWS = 53  # 행 (Y축)
N_COLS = 25  # 열 (X축)
MIN_VALUE = -2.0 
MAX_VALUE = 4.0  
CENTER_VALUE = 0.0

# 2. 색상 클러스터링 기반 변환 함수 (색상표 시각 분석 기반 미세 조정 - 최종 버전)
def rgb_to_value_final_tuned(r_norm, g_norm, b_norm):
    """
    제공된 색상표(Color Bar) 이미지의 시각적 특징을 반영하여 5단계 매핑 경계를 미세 조정합니다.
    (이 함수는 메인 함수에서 'rgb_to_value_final_tuned' 이름으로 호출되어야 합니다.)
    """
    
    # ... (생략: 1~3단계 빨강, 주황, 노랑 로직은 이전과 동일)

    # 1. 빨강 (4.0 ~ 3.0)
    if r_norm > 0.65 and g_norm < 0.35:
        value = 3.0 + (r_norm - 0.65) * 3.0 
        return np.clip(value, 3.0, 4.0)

    # 2. 주황 (3.0 ~ 2.0)
    elif r_norm > 0.4 and g_norm > 0.4 and b_norm < 0.25:
        value = 3.0 - b_norm * (1.0 / 0.25) 
        return np.clip(value, 2.0, 3.0)

    # 3. 노랑 ~ 연두 (2.0 ~ 1.0)
    elif r_norm > 0.5 and g_norm > 0.5 and b_norm < 0.5:
        value = 3.0 - b_norm * (2.0 / 0.25) 
        value = value + (1.0 - abs(r_norm - g_norm)) * 0.5 
        return np.clip(value, 1.0, 2.0)
        
    # 4. 하늘색 (1.0 ~ 0.0) 
    elif b_norm > 0.4 and (r_norm + g_norm) / 2 < b_norm and b_norm < 0.75:
        value = 1.0 - (b_norm - 0.45) * (1.0 / 0.3) 
        return np.clip(value, 0.0, 1.0)
        
    # 5. 진한 파랑 (0.0 ~ -2.0) 
    else:
        value = 0.0 - (b_norm - 0.75) * (2.0 / 0.25) 
        return np.clip(value, -2.0, 0.0)


# 3. 이미지 분석 및 수치 추출 메인 함수
def quantify_heatmap(image_path, rows, cols, vmin, vmax, center):
    
    quantified_data = np.zeros((rows, cols))
    
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {image_path}")
            
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # ⚠️ [가장 중요한 수동 조정] 픽셀 좌표 설정 ⚠️
        x_start = 28  
        x_end = 478   
        y_start = 28  
        y_end = 650   
        # ----------------------------------------------------
        
        cropped_img = img_rgb[y_start:y_end, x_start:x_end]
        cropped_height, cropped_width, _ = cropped_img.shape
        
        cell_height = cropped_height // rows
        cell_width = cropped_width // cols
        
        # 각 셀을 순회하며 평균 색상 추출 및 값 변환
        for i in range(rows):
            for j in range(cols):
                border_trim_y = int(cell_height * 0.15)
                border_trim_x = int(cell_width * 0.15)

                sampled_area = cropped_img[
                    i * cell_height + border_trim_y : (i + 1) * cell_height - border_trim_y,
                    j * cell_width + border_trim_x : (j + 1) * cell_width - border_trim_x
                ]
                
                avg_rgb_255 = np.mean(sampled_area, axis=(0, 1))
                r, g, b = avg_rgb_255 / 255.0 # 0~1로 정규화
                
                # 📢 수정됨: 정확한 함수 이름 호출 📢
                value = rgb_to_value_final_tuned(r, g, b) 
                quantified_data[i, j] = value
        
    except Exception as e:
        print(f"❌ 이미지 분석 또는 파일 처리 중 오류 발생: {e}")
        return np.zeros((rows, cols))
        
    return quantified_data


# 4. 수치화 실행 및 엑셀 저장
final_data = quantify_heatmap(HEATMAP_PATH, N_ROWS, N_COLS, MIN_VALUE, MAX_VALUE, CENTER_VALUE)

if final_data.any():
    df_result = pd.DataFrame(final_data).round(3)
    try:
        df_result.to_excel(EXCEL_FILENAME, index=False, header=False)
        print(f"\n🎉 **최종 데이터가 '{EXCEL_FILENAME}' 파일로 저장되었습니다.**")
        print(f"저장된 데이터 크기: {N_ROWS}행, {N_COLS}열")
    except Exception as e:
        print(f"❌ 엑셀 파일 저장 중 오류가 발생했습니다: {e}")


🎉 **최종 데이터가 'heatmap_qua.xlsx' 파일로 저장되었습니다.**
저장된 데이터 크기: 53행, 25열


In [58]:
df=pd.read_excel("mix pure result.xlsx")
df

,Unnamed: 0,N (mixed-breed/purebred),"Mixed-breed Prevalence (%; n = 13,923)","Purebred Prevalence (%; n = 13,618)",p-value
0,No ORMC,2888/3031,20.74,22.26,0.002
1,Dental calculus,2089/2011,15.00,14.77,0.058
2,Extracted teeth,1877/810,13.48,5.95,1.66 × 10 −98
3,Dog bite,1663/1305,11.94,9.58,2.64 × 10 −10
4,Fractured teeth,997/810,7.16,5.95,4.82 × 10 −5
...,...,...,...,...,...
59,Osteoarthritis,43,7.07,5.29–9.39,5.95 × 10−5
60,Fractured teeth,39,6.41,4.73–8.65,0.085
61,Pruritis (itchy skin),34,5.59,4.03–7.71,0.009
62,Sebaceous cysts,33,5.43,3.89–7.52,3.96 × 10−5


In [62]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

# --- 파일 경로 (고객님 요청대로 설정) ---
LOGOR_FILE = 'Table_1.XLSX' 
REF_PREVALENCE_FILE = 'mix pure result.xlsx'
OUTPUT_FILE = 'Heatmap_Prevalence_Result_Final.xlsx' 

try:
    # 1. logOR 데이터 불러오기 (Table_1.XLSX)
    # 고객님이 직접 확인하신 Header Index 0을 사용하여 읽습니다.
    df_logor = pd.read_excel(LOGOR_FILE, header=0)
    required_logor_cols = ['Medical condition', 'Breed', 'beta (logOR)']

    # 컬럼이 있는지 최종 확인
    if not all(col in df_logor.columns for col in required_logor_cols):
         # 헤더 0에서 컬럼을 찾지 못했다면, 원본 파일 구조에 맞게 헤더를 다시 시도합니다. (Header 2)
        df_logor = pd.read_excel(LOGOR_FILE, header=2)
        if not all(col in df_logor.columns for col in required_logor_cols):
             raise ValueError(f"'{LOGOR_FILE}' 파일에서 필요한 컬럼({required_logor_cols})을 찾을 수 없습니다.")

    df_logor = df_logor[required_logor_cols].copy()
    
    # 2. 기준 유병률 데이터 불러오기 (mix pure result.xlsx)
    # 고객님의 성공 경험을 바탕으로 Header Index 0으로 읽습니다.
    df_ref = pd.read_excel(REF_PREVALENCE_FILE, header=0)

    # 컬럼 이름 유연하게 찾기
    ref_col_name = 'Purebred Prevalence'
    if ref_col_name not in df_ref.columns:
        # 'Prevalence'가 포함된 컬럼을 찾습니다.
        potential_ref_col = [col for col in df_ref.columns if 'Prevalence' in str(col)][0]
        ref_col_name = potential_ref_col
        
    df_ref = df_ref[['Medical condition', ref_col_name]].copy()
    df_ref.columns = ['Medical condition', 'P_ref_percent']
    
    # --- 3. 데이터 병합 및 변환 (논리 유지) ---
    df_merged = pd.merge(df_logor, df_ref, on='Medical condition', how='left')
    
    # 4. logOR을 유병률(%)로 변환하는 함수 정의
    def logor_to_prevalence(row):
        logor = row['beta (logOR)']
        p_ref_percent = row['P_ref_percent']
        
        if pd.isna(logor) or pd.isna(p_ref_percent): return np.nan
        P_ref = p_ref_percent / 100 
        
        if P_ref >= 1.0: return 100.0
        elif P_ref <= 0.0: return 0.0

        Odds_ref = P_ref / (1 - P_ref)
        Odds_breed = np.exp(logor) * Odds_ref
        P_breed = (Odds_breed / (1 + Odds_breed)) * 100
        
        return np.clip(P_breed, 0.0, 100.0)

    # 5. 변환 함수 적용 및 엑셀 저장
    df_merged['Estimated Prevalence (%)'] = df_merged.apply(logor_to_prevalence, axis=1).round(3)
    df_pivot = df_merged.pivot(index='Medical condition', columns='Breed', values='Estimated Prevalence (%)')

    df_pivot.to_excel(OUTPUT_FILE)
    
    print(f"\n🎉 **최종 변환 성공! $0\%$ ~ $100\%$로 변환된 유병률 행렬이 '{OUTPUT_FILE}' 파일로 저장되었습니다.**")
    print(f"저장된 데이터 크기: {len(df_pivot)}행, {len(df_pivot.columns)}열")

except Exception as e:
    print(f"❌ 데이터 처리 중 예상치 못한 오류 발생: {e}")
    print("⚠️ 파일 읽기 자체는 성공했으나, 컬럼 이름(특히 'beta (logOR)'나 'Purebred Prevalence')을 찾는 데 문제가 발생했습니다. 엑셀 파일 내의 실제 컬럼 이름을 다시 한 번 확인해주세요.")

❌ 데이터 처리 중 예상치 못한 오류 발생: "['Medical condition'] not in index"
⚠️ 파일 읽기 자체는 성공했으나, 컬럼 이름(특히 'beta (logOR)'나 'Purebred Prevalence')을 찾는 데 문제가 발생했습니다. 엑셀 파일 내의 실제 컬럼 이름을 다시 한 번 확인해주세요.


In [ ]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 무시 설정
warnings.filterwarnings("ignore")

# --- 파일 경로 설정 (업로드된 원본 파일 이름 사용) ---
LOGOR_FILE = 'Table_1.XLSX' 
REF_PREVALENCE_FILE = 'mix pure result.xlsx'
OUTPUT_FILE = 'Heatmap_Prevalence_Result_Final.xlsx' 

df_logor=pd.read_excel(LOGOR_FILE)

try:
    # 1. logOR 데이터 불러오기 
    required_logor_cols = ['Medical condition', 'Breed', 'beta (logOR)']
    df_logor = robust_file_reader(LOGOR_FILE, required_logor_cols)

    if df_logor is None:
        raise ValueError(f"'{LOGOR_FILE}' 파일에서 필요한 컬럼({required_logor_cols})을 찾을 수 없습니다. 파일 형식을 확인해주세요.")
        
    df_logor = df_logor[required_logor_cols].copy()
    
    # 2. 기준 유병률 데이터 불러오기
    df_ref = robust_file_reader(REF_PREVALENCE_FILE, ['Medical condition']) 

    if df_ref is None:
        raise ValueError(f"'{REF_PREVALENCE_FILE}' 파일을 읽을 수 없습니다. 파일 형식을 확인해주세요.")
        
    # 'Purebred Prevalence' 컬럼 찾기 (이름이 다를 경우 유연하게 처리)
    ref_col_name = 'Purebred Prevalence (%; n=13,618)'
    if ref_col_name not in df_ref.columns:
        potential_ref_col = [col for col in df_ref.columns if 'Purebred Prevalence' in str(col)][0]
        ref_col_name = potential_ref_col
        
    df_ref = df_ref[['Medical condition', ref_col_name]].copy()
    df_ref.columns = ['Medical condition', 'P_ref_percent']
    
    # --- 3. 데이터 병합 ---
    df_merged = pd.merge(df_logor, df_ref, on='Medical condition', how='left')
    
    # --- 4. logOR을 유병률(%)로 변환하는 함수 정의 ---
    def logor_to_prevalence(row):
        logor = row['beta (logOR)']
        p_ref_percent = row['P_ref_percent']
        
        # NaN 값 처리
        if pd.isna(logor) or pd.isna(p_ref_percent): 
            return np.nan
        
        P_ref = p_ref_percent / 100 
        
        # 0% 또는 100% 예외 처리
        if P_ref >= 1.0: return 100.0
        elif P_ref <= 0.0: return 0.0

        # A. 기준 Odds 계산: Odds = P / (1 - P)
        Odds_ref = P_ref / (1 - P_ref)
        
        # B. 품종 Odds 계산: Odds_breed = e^logOR * Odds_ref
        Odds_breed = np.exp(logor) * Odds_ref
        
        # C. 품종 유병률(%) 계산: P_breed = Odds_breed / (1 + Odds_breed)
        P_breed = (Odds_breed / (1 + Odds_breed)) * 100
        
        return np.clip(P_breed, 0.0, 100.0) # 0% ~ 100% 범위로 제한

    # 5. 변환 함수 적용 및 피벗 테이블 생성
    df_merged['Estimated Prevalence (%)'] = df_merged.apply(logor_to_prevalence, axis=1).round(3)
    df_pivot = df_merged.pivot(index='Medical condition', columns='Breed', values='Estimated Prevalence (%)')

    # 6. 최종 엑셀 파일로 저장
    df_pivot.to_excel(OUTPUT_FILE)
    
    print(f"\n🎉 **$\beta(\text{logOR})$ 값이 $0\%$ ~ $100\%$ 유병률로 변환되어 '{OUTPUT_FILE}' 파일로 저장되었습니다.**")
    print(f"저장된 데이터 크기: {len(df_pivot)}행, {len(df_pivot.columns)}열")

except Exception as e:
    print(f"❌ 데이터 처리 중 예상치 못한 오류 발생: {e}")

❌ 데이터 처리 중 예상치 못한 오류 발생: 'mix pure result.xlsx' 파일을 읽을 수 없습니다. 파일 형식을 확인해주세요.


In [67]:
import pandas as pd
import numpy as np
import warnings

# 경고 메시지 무시 설정
warnings.filterwarnings("ignore")

# --- 파일 경로 설정 ---
LOGOR_FILE = 'Table_1.XLSX - each breed vs all by condition .csv' 
REF_PREVALENCE_FILE = 'mix pure result.xlsx - Sheet1.csv'
OUTPUT_FILE = 'Heatmap_Prevalence_Result_Final.xlsx' 

try:
    # 1. LogOR 데이터 불러오기 (컬럼 이름 대신 인덱스 0, 1, 2 사용)
    # 헤더는 4번째 줄 (index 3)로 지정합니다.
    df_logor = pd.read_csv(LOGOR_FILE, header=3, usecols=[0, 1, 2], encoding='utf-8')
    df_logor.columns = ['Medical condition', 'Breed', 'beta (logOR)'] # 병합을 위해 임시 컬럼명 지정
    
    # 2. 기준 유병률 데이터 불러오기 (컬럼 이름 대신 인덱스 0, 3 사용)
    # 헤더는 1번째 줄 (index 0)로 지정합니다.
    df_ref = pd.read_csv(REF_PREVALENCE_FILE, header=0, usecols=[0, 3], encoding='utf-8')
    df_ref.columns = ['Medical condition', 'P_ref_percent'] # 병합 및 계산을 위해 임시 컬럼명 지정

    # --- 3. 데이터 병합 ---
    # Medical condition 기준으로 병합
    df_merged = pd.merge(df_logor, df_ref, on='Medical condition', how='left')
    
    # --- 4. LogOR을 유병률(%)로 변환하는 함수 정의 ---
    def logor_to_prevalence(row):
        logor = row['beta (logOR)']
        p_ref_percent = row['P_ref_percent']
        
        # NaN 값 처리
        if pd.isna(logor) or pd.isna(p_ref_percent): 
            return np.nan
        
        # 유병률(%)을 비율(0~1)로 변환
        P_ref = p_ref_percent / 100 
        
        # 0% 또는 100% 예외 처리 (수렴 방지)
        if P_ref >= 1.0: return 100.0
        elif P_ref <= 0.0: return 0.0

        # A. 기준 Odds 계산: Odds_ref = P_ref / (1 - P_ref)
        Odds_ref = P_ref / (1 - P_ref)
        
        # B. 품종 Odds 계산: Odds_breed = e^logOR * Odds_ref
        Odds_breed = np.exp(logor) * Odds_ref
        
        # C. 품종 유병률(%) 계산: P_breed = Odds_breed / (1 + Odds_breed) * 100
        P_breed = (Odds_breed / (1 + Odds_breed)) * 100
        
        return np.clip(P_breed, 0.0, 100.0) # 0% ~ 100% 범위로 제한

    # 5. 변환 함수 적용 및 피벗 테이블 생성
    df_merged['Estimated Prevalence (%)'] = df_merged.apply(logor_to_prevalence, axis=1).round(3)
    
    # 품종별 발병률 행렬(Heatmap 형태)로 변환
    df_pivot = df_merged.pivot(index='Medical condition', columns='Breed', values='Estimated Prevalence (%)')

    # 6. 최종 엑셀 파일로 저장
    df_pivot.to_excel(OUTPUT_FILE)
    
    print(f"\n🎉 **컬럼 인덱스를 사용하여 $\\beta(\\text{{logOR}})$ 값이 $0\\% \\sim 100\\%$ 유병률로 변환되어 '{OUTPUT_FILE}' 파일로 저장되었습니다.**")
    print(f"저장된 데이터 크기: {len(df_pivot)}행, {len(df_pivot.columns)}열")

except Exception as e:
    print(f"❌ 데이터 처리 중 오류 발생: {e}")
    print("⚠️ 컬럼 인덱스를 명시했음에도 오류가 발생했습니다. 두 파일의 'Medical condition'과 'Purebred Prevalence' 컬럼의 위치(인덱스)를 다시 한 번 확인해 주시면 수정하겠습니다.")

❌ 데이터 처리 중 오류 발생: [Errno 2] No such file or directory: 'Table_1.XLSX - each breed vs all by condition .csv'
⚠️ 컬럼 인덱스를 명시했음에도 오류가 발생했습니다. 두 파일의 'Medical condition'과 'Purebred Prevalence' 컬럼의 위치(인덱스)를 다시 한 번 확인해 주시면 수정하겠습니다.
